In [7]:
import pandas as pd
import os
import json
from DB_helpers import open_connection
from PATH_CONFIGS import DB_CONFIG_FILE_NAME

In [9]:
db_config = os.path.join('/Users/aruscha/Documents/cut-parkplatz-neu/add-files/', DB_CONFIG_FILE_NAME)
db_user = '/Users/aruscha/Documents/cut-parkplatz-neu/add-files/db_aruscha.json'
validation_file = '/Users/aruscha/Documents/cut-parkplatz-neu/add-files/VALIDATION_suedvo_segment_sides.geojson'
# Opening JSON file
f = open(validation_file)
 
# returns JSON object as a dictionary
data = json.load(f)

df = pd.json_normalize([feat['properties'] for feat in data['features']])
df['parking'] = df['parking'].replace('diagaonal/senkrecht', 'diagonal/senkrecht')
df['side'] = df['side'].replace('Left', 'left')
df['side'] = df['side'].replace('Right', 'right')


print(df.columns)
df.drop('X', axis=1, inplace=True)

#print(f"The validation df has {len(df)} entries")
same_count, different_count = 0, 0

with open_connection(db_config, db_user) as con:
    cursor = con.cursor()
    cursor.execute("""SELECT * FROM PARKING""")
    res = cursor.fetchall()
    for item in res:

        segment_id = item[0]
        segmentation_number = item[1]
        parking_side = item[2]
        parking_value = item[3]

        df_parking_entry = df[(df['segment_id'] == segment_id) & (df['segmentation_number'] == segmentation_number) & (df['side'] == parking_side)]
        if not df_parking_entry.empty:

            df_parking_entry = df_parking_entry['parking'].iloc[0]
            print(f"checking segment id: {segment_id}, cyclo: {parking_value}, manual: {df_parking_entry} ")

            if str(df_parking_entry) == str(parking_value):
                same_count += 1
            else:
                different_count += 1

print("end")
print("total items: ", len(res))
print("SAME items: ", same_count, "percentage: ", round(same_count/len(res)*100, 2))
print("different items:", different_count, "percentage: ", round(different_count/len(res)*100, 2))

Index(['X', 'segment_id', 'segmentation_number', 'side', 'parking'], dtype='object')
checking segment id: 2872, cyclo: parallel, manual: parallel 
checking segment id: 2872, cyclo: kein Auto, manual: kein Auto 
checking segment id: 2894, cyclo: diagonal/senkrecht, manual: diagonal/senkrecht 
checking segment id: 2894, cyclo: parallel, manual: parallel 
checking segment id: 2904, cyclo: parallel, manual: parallel 
checking segment id: 2904, cyclo: kein Auto, manual: kein Auto 
checking segment id: 2905, cyclo: diagonal/senkrecht, manual: diagonal/senkrecht 
checking segment id: 2905, cyclo: parallel, manual: parallel 
checking segment id: 2906, cyclo: diagonal/senkrecht, manual: diagonal/senkrecht 
checking segment id: 2906, cyclo: parallel, manual: parallel 
checking segment id: 2908, cyclo: parallel, manual: parallel 
checking segment id: 2908, cyclo: parallel, manual: parallel 
checking segment id: 2911, cyclo: diagonal/senkrecht, manual: diagonal/senkrecht 
checking segment id: 2911